In [102]:
from IPython.display import display, HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

In [1]:
import findspark
findspark.init()


In [2]:
from pyspark.sql import SparkSession


In [3]:
spark=SparkSession.builder.getOrCreate()

25/03/20 15:15:47 WARN Utils: Your hostname, sherif-Virtual-Machine resolves to a loopback address: 127.0.1.1; using 172.29.81.158 instead (on interface eth0)
25/03/20 15:15:47 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/20 15:15:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
spark

In [6]:
sc=spark.sparkContext

In [7]:
from pyspark.sql.functions import *

In [8]:
from pyspark.sql.types import *

In [16]:
my_schema = StructType([
    StructField('FName', StringType(), False),  # First Name (String, Not Nullable)
    StructField('LName', StringType(), True),    # Last Name (String, Nullable)
    StructField('Age', IntegerType(), False),   # Age (Integer, Not Nullable)
    StructField('Address', StringType(), False),# Address (String, Not Nullable)
    StructField('Gender', StringType(), False)  # Gender (String, Not Nullable)
])

In [17]:
from faker import Faker
fake=Faker()

data=[]

for _ in range(50):  #create 50 sample of th data
    first_name=fake.first_name()
    last_name=fake.last_name()
    age=fake.random_int(min=18,max=65) ## create random age between 18 and 65
    address=fake.address().replace('\n',', ') ## replace new line with comma
    gender=fake.random_element(elements=('M','F'))

    data.append((first_name,last_name,age,address,gender))

In [18]:
ex_data=spark.createDataFrame(data,my_schema)

In [20]:
ex_data.show(5,truncate=False)

+--------+---------+---+---------------------------------------------------+------+
|FName   |LName    |Age|Address                                            |Gender|
+--------+---------+---+---------------------------------------------------+------+
|Laura   |Rodriguez|36 |7292 Judy Lodge Suite 999, Lake Shawnland, ME 90715|M     |
|Patricia|Gomez    |48 |028 Lauren Lane Suite 386, North Brandon, SD 07741 |M     |
|Kathleen|Odom     |27 |99831 Lee Walk, Lake Danielstad, UT 11777          |F     |
|Nancy   |Grant    |31 |18454 Mary Fort, Caseyburgh, MA 30110              |F     |
|Oscar   |Clark    |19 |5959 Logan Circles, Kristenstad, MA 46428          |F     |
+--------+---------+---+---------------------------------------------------+------+
only showing top 5 rows



In [26]:
ex1=ex_data.sort(desc('FName'))
ex1.show(3)

+--------+---------+---+--------------------+------+
|   FName|    LName|Age|             Address|Gender|
+--------+---------+---+--------------------+------+
|Veronica|Frederick| 41|36193 Tracy Club ...|     F|
|   Tracy| Jennings| 61|81448 Kevin Ville...|     F|
|    Tina| Lawrence| 20|720 Barnes Vista ...|     F|
+--------+---------+---+--------------------+------+
only showing top 3 rows



In [29]:
ex1=ex_data.sort('FName',ascending=False)
ex1.show(3)

+--------+---------+---+--------------------+------+
|   FName|    LName|Age|             Address|Gender|
+--------+---------+---+--------------------+------+
|Veronica|Frederick| 41|36193 Tracy Club ...|     F|
|   Tracy| Jennings| 61|81448 Kevin Ville...|     F|
|    Tina| Lawrence| 20|720 Barnes Vista ...|     F|
+--------+---------+---+--------------------+------+
only showing top 3 rows



### Read csv file and deal with null values

In [34]:
ndata=spark.read.csv('/home/sherif/Downloads/data/NullData.csv',header=True,inferSchema=True)
ndata.show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| NULL|
|emp2| NULL| NULL|
|emp3| NULL|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [35]:
ndata.printSchema()

root
 |-- Id: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sales: double (nullable = true)



### Drop null values

In [43]:
n1=ndata.na.drop()
n1.show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp4|Cindy|456.0|
+----+-----+-----+



In [48]:
## the default is drop all rows if any column is null
n2=ndata.na.drop(how='all')
n2.show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| NULL|
|emp2| NULL| NULL|
|emp3| NULL|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [49]:
#if 2 column is not null don't drop else drop
n3=ndata.na.drop(thresh=2)
n3.show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| NULL|
|emp3| NULL|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [51]:
## specific columns 
n4=ndata.na.drop(subset=['Name','Sales'],how='all')
n4.show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| NULL|
|emp3| NULL|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



### fill na

In [52]:

n5=ndata.na.fill('new name')
n5.show()

+----+--------+-----+
|  Id|    Name|Sales|
+----+--------+-----+
|emp1|    John| NULL|
|emp2|new name| NULL|
|emp3|new name|345.0|
|emp4|   Cindy|456.0|
+----+--------+-----+



In [53]:

n6=ndata.na.fill(0)
n6.show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|  0.0|
|emp2| NULL|  0.0|
|emp3| NULL|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



``` last examples it depends on schema to fill specific columns based on cou\lumns data types now we can use subset to modify specific columns manaully ```

In [54]:
n7=ndata.na.fill('new',subset=['Name'])
n7.show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| NULL|
|emp2|  new| NULL|
|emp3|  new|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



### fill na with mean values

In [86]:
m1=ndata.select(mean(col('Sales')).alias('m')).collect()
m=m1[0][0]

In [87]:
n1=ndata.select(mean(ndata['Sales']).alias('mean_Sales')).collect()
n1[0][0]

400.5

In [89]:
fill_mean=ndata.fillna(m,subset='Sales')
fill_mean.show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|400.5|
|emp2| NULL|400.5|
|emp3| NULL|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



### read csv file and use sampling ratio to reduce the waste time

In [103]:
df=spark.read.csv('/home/sherif/Downloads/data/sf-fire-calls.csv',header=True,
                  inferSchema=True,
                  samplingRatio=0.001)

In [104]:
df.printSchema()

root
 |-- CallNumber: integer (nullable = true)
 |-- UnitID: string (nullable = true)
 |-- IncidentNumber: integer (nullable = true)
 |-- CallType: string (nullable = true)
 |-- CallDate: string (nullable = true)
 |-- WatchDate: string (nullable = true)
 |-- CallFinalDisposition: string (nullable = true)
 |-- AvailableDtTm: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Zipcode: integer (nullable = true)
 |-- Battalion: string (nullable = true)
 |-- StationArea: string (nullable = true)
 |-- Box: integer (nullable = true)
 |-- OriginalPriority: string (nullable = true)
 |-- Priority: string (nullable = true)
 |-- FinalPriority: integer (nullable = true)
 |-- ALSUnit: boolean (nullable = true)
 |-- CallTypeGroup: string (nullable = true)
 |-- NumAlarms: integer (nullable = true)
 |-- UnitType: string (nullable = true)
 |-- UnitSequenceInCallDispatch: integer (nullable = true)
 |-- FirePreventionDistrict: string (nullable = true)


In [106]:
df.show(3)

+----------+------+--------------+----------------+----------+----------+--------------------+--------------------+--------------------+----+-------+---------+-----------+----+----------------+--------+-------------+-------+-------------+---------+--------+--------------------------+----------------------+------------------+--------------------+--------------------+-------------+---------+
|CallNumber|UnitID|IncidentNumber|        CallType|  CallDate| WatchDate|CallFinalDisposition|       AvailableDtTm|             Address|City|Zipcode|Battalion|StationArea| Box|OriginalPriority|Priority|FinalPriority|ALSUnit|CallTypeGroup|NumAlarms|UnitType|UnitSequenceInCallDispatch|FirePreventionDistrict|SupervisorDistrict|        Neighborhood|            Location|        RowID|    Delay|
+----------+------+--------------+----------------+----------+----------+--------------------+--------------------+--------------------+----+-------+---------+-----------+----+----------------+--------+------------

In [107]:
description=df.describe()

In [108]:
description.printSchema()

root
 |-- summary: string (nullable = true)
 |-- CallNumber: string (nullable = true)
 |-- UnitID: string (nullable = true)
 |-- IncidentNumber: string (nullable = true)
 |-- CallType: string (nullable = true)
 |-- CallDate: string (nullable = true)
 |-- WatchDate: string (nullable = true)
 |-- CallFinalDisposition: string (nullable = true)
 |-- AvailableDtTm: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Zipcode: string (nullable = true)
 |-- Battalion: string (nullable = true)
 |-- StationArea: string (nullable = true)
 |-- Box: string (nullable = true)
 |-- OriginalPriority: string (nullable = true)
 |-- Priority: string (nullable = true)
 |-- FinalPriority: string (nullable = true)
 |-- CallTypeGroup: string (nullable = true)
 |-- NumAlarms: string (nullable = true)
 |-- UnitType: string (nullable = true)
 |-- UnitSequenceInCallDispatch: string (nullable = true)
 |-- FirePreventionDistrict: string (nullable = true)
 |-- Sup

In [109]:
description.show()

+-------+--------------------+-----------------+-----------------+--------------------+----------+----------+--------------------+--------------------+--------------------+-----------+------------------+---------+------------------+------------------+------------------+------------------+------------------+--------------------+-------------------+--------+--------------------------+----------------------+------------------+--------------------+--------------------+-------------+-----------------+
|summary|          CallNumber|           UnitID|   IncidentNumber|            CallType|  CallDate| WatchDate|CallFinalDisposition|       AvailableDtTm|             Address|       City|           Zipcode|Battalion|       StationArea|               Box|  OriginalPriority|          Priority|     FinalPriority|       CallTypeGroup|          NumAlarms|UnitType|UnitSequenceInCallDispatch|FirePreventionDistrict|SupervisorDistrict|        Neighborhood|            Location|        RowID|            Del